Скачиваем компилятор

In [ ]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-7v7ajipr
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-7v7ajipr
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=4afcb61eb53eb3b94e1ddc6732953f0aa73170c0bb32f4002453e84d9036f1c0
  Stored in directory: /tmp/pip-ephem-wheel-cache-k5urkpe_/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


Непосредственно программа

In [ ]:
%%cu
#include <cublas_v2.h>
#include <malloc.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <stdexcept>
using namespace std;
/*
 Редукция сложением массива inArray
 Ответ будет лежать в outArray[0]
*/
__global__ void kernel(double* inArray, double* outArray, int n) {

    extern __shared__ double sharedArray[];

    // считаем свой id и шаг суммирования во всем массиве
    int grid_tid = blockIdx.x*blockDim.x+threadIdx.x;
    int grid_stride = blockDim.x * gridDim.x;

    // свой id внутри блока
    int block_tid = threadIdx.x;

    // сумма своих элементов "вне блока"
    double firstSum = 0;

    int i = grid_tid;
    firstSum = inArray[i] + inArray[i+grid_stride];
    i += grid_stride<<1;
    firstSum += i < n ? inArray[i] : 0;
    firstSum += i+grid_stride < n ? inArray[i+grid_stride] : 0;

    sharedArray[block_tid] = firstSum;
    __syncthreads();
    // теперь в sharedArray всплошную лежат промежуточные суммы

    // теперь мы работаем только в пределах своего блока с внутренней памятью
    int block_stride = blockDim.x>>1;
    // будем суммировать пока не окажемся в половине sharedArray из которой берут промежуточные суммы
    while (block_tid < block_stride) {
        sharedArray[block_tid] += sharedArray[block_tid + block_stride];
        //и уменьшаем шаг на 2
        block_stride >>= 1;

         __syncthreads();
    }
    // используем идентификатор блока (тк в каждом блоке "до конца" дойдет только один поток - с номером 0)
    // чтобы в outArray итоговые-промежуточные суммы лежили всплошную
    outArray[blockIdx.x] = sharedArray[0];
}

void count_cuda_dims(int& blocksPerGrid, int& threadsPerBlock, int n) {
    const int MAX_THREADS_PER_BLOCK = 1024;

    //сколько операций нужно выполнить в первую итерацию
    double numOperations = (float)n / 2;

    //степень двойки, при возведении в которую получим наибольшее число меньшее numOperations
    double nearestPowLower = log2(numOperations);
    int intPower = floor(nearestPowLower);
    //получим опитмальное кол-во потоков (ближайшая меньшая степень двойки к кол-ву элементов/2)
    int numThreads = 1 << intPower;

    //максимум потоков в блоке - 1024
    threadsPerBlock = numThreads < MAX_THREADS_PER_BLOCK ? numThreads : MAX_THREADS_PER_BLOCK;
    //тут ничего округлять не надо - точно делится нацело (numThreads у нас ведь степень двойки)
    blocksPerGrid = numThreads < MAX_THREADS_PER_BLOCK ? 1 : numThreads / threadsPerBlock;
}

void handle_cuda_result(cudaError_t cuerr, char msg[]) {
    if (cuerr != cudaSuccess) {
        fprintf(stderr, cudaGetErrorString(cuerr));
        fprintf(stderr, "\n");
        throw runtime_error(msg);
    }
}

/*
 Загрузка данных на GPU
 @ resultGpuPointer - перезаписывается, указатель на указатель на память GPU (чтобы значение указателя сохранялось)
 @ a - указатель на массив
 @ size - размер массива
*/
void upload_to_device(double** resultGpuPointer, double* a, int size) {
    int sizeInBytes = size * sizeof(double);
    //выделяем память
    cudaError_t cuerr = cudaMalloc((void**)resultGpuPointer, sizeInBytes);
    handle_cuda_result(cuerr, "Cannot allocate device array");

    // копируем массив
    //*resultGpuPointer - разыменовываем указатель на указатель, чтобы передать указатель
    cuerr = cudaMemcpy(*resultGpuPointer, a, sizeInBytes, cudaMemcpyHostToDevice);
    handle_cuda_result(cuerr, "Cannot copy a array from host to device");
}

/*
 Выделение памяти на GPU
 @ resultGpuPointer - перезаписывается, указатель на указатель на память GPU (чтобы значение указателя сохранялось)
 @ size - размер массива
*/
void allocate_memory(double** resultGpuPointer,  int size) {
    int sizeInBytes = size * sizeof(double);
    //выделяем память
    cudaError_t cuerr = cudaMalloc((void**)resultGpuPointer, sizeInBytes);
    handle_cuda_result(cuerr, "Cannot allocate device array");

}

/*
Загрузка данных на хост-машину с GPU
 @ gpuPointer - указатель на память GPU
 @ resultA - указатель на результирующий массив (память должная быть выделена)
 @ size - размер массива
*/
void download_from_device(double* gpuPointer, double* resultA, int size) {
    int sizeInBytes = size * sizeof(double);
    // копируем массив
    cudaError_t cuerr = cudaMemcpy(resultA, gpuPointer, sizeInBytes, cudaMemcpyDeviceToHost);
    handle_cuda_result(cuerr, "Cannot copy a array from device to host");
}

/*
  Редукция сложением
*/
double reduce_gpu(double* inArray, double* outArray, int n){
    cudaError_t cuerr;

    int threadsPerBlock;
    int blocksPerGrid;

    count_cuda_dims(blocksPerGrid, threadsPerBlock, n);

    // Выделение памяти на устройстве
    double* inGpuPointer = NULL;
    //передаем указатель на указатель, чтобы работать со значением указателя                                                                                                                ///
    upload_to_device(&inGpuPointer, inArray, n);

    double* outGpuPointer = NULL;
    //нужно выделить память только под количество потоков, тк там будут лежать промежуточные суммы
    allocate_memory(&outGpuPointer, blocksPerGrid*threadsPerBlock);

    // Создание обработчиков событий
    cudaEvent_t start, stop;
    float gpuTime = 0.0f;
    cuerr = cudaEventCreate(&start);
    handle_cuda_result(cuerr, "Cannot create CUDA start event");

    cuerr = cudaEventCreate(&stop);
    handle_cuda_result(cuerr, "Cannot create CUDA stop event");

    cuerr = cudaEventRecord(start, 0);
    handle_cuda_result(cuerr, "Cannot record CUDA start event");


    double* pointerBuf;
    //Запуск ядра
    while(n > 1) {
        kernel <<< blocksPerGrid, threadsPerBlock, threadsPerBlock * sizeof(double) >>> (inGpuPointer, outGpuPointer, n);
        // после частичной редукции от каждого блока останется одина итоговая-промежуточная сумма
        n = blocksPerGrid;
        // считаем заново параметры с обновленным n
        count_cuda_dims(blocksPerGrid, threadsPerBlock, n);

        // меняем местами входной и выходной массив и не читать-писать в один и тот же, чтобы избежать dirty-read
        // в них обоих лежит мусор на незначимых для нас местах (нам интересны только первые blocksPerGrid элементов)
        pointerBuf = inGpuPointer;
        inGpuPointer = outGpuPointer;
        outGpuPointer = pointerBuf;
    }

    handle_cuda_result(cudaGetLastError(), "Cannot launch CUDA kernel");

    // Синхронизация устройств
    cuerr = cudaDeviceSynchronize();
    handle_cuda_result(cudaGetLastError(), "Cannot synchronize CUDA kernel");

    // Установка точки окончания
    cuerr = cudaEventRecord(stop, 0);
    handle_cuda_result(cuerr, "Cannot record CUDA stop event");

    // Копирование результата на хост, интересует только 1 элемент, где есть результат
    download_from_device(inGpuPointer, outArray, 1);

    cuerr = cudaEventElapsedTime(&gpuTime, start, stop);
    handle_cuda_result(cuerr, "Cannot get elapsed time");
    double time = gpuTime / 1000;

    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(inGpuPointer);
    cudaFree(outGpuPointer);
    return time;
}

double reduce_cpu(double* inArray, double* outArray,  int n){
    double sum = 0;
    double time = clock();

    for(int i = 0; i < n; ++i) {
        sum += inArray[i];
    }

    time = clock() - time;
    time/=CLOCKS_PER_SEC;

    outArray[0] = sum;
    return time;
}

void fill_array(double* a, int n) {
    srand (time(NULL));
    for(int i = 0; i < n; ++i) {
        a[i] = rand()/1000000000.0;
    }

}

void check(double* cpu, double* gpu,  double precision) {
    if (fabs(cpu[0] - gpu[0]) > precision) {
        fprintf(stderr, "Ответы не равны: cpu=%f; gpu=%f\n", cpu[0], gpu[0]);
        throw runtime_error("Ответы не равны");
    }
}

int main(int argc, char* argv[]) {
    int n = 1000000;
    double* a = (double*) malloc(n*sizeof(double));
    double* result_gpu = (double*) malloc(sizeof(double));
    double* result_cpu = (double*) malloc(sizeof(double));

    fill_array(a, n);

    double time_gpu = reduce_gpu(a,result_gpu, n);
    printf("result gpu: %f, time gpu: %f\n",result_gpu[0], time_gpu);

    double time_cpu = reduce_cpu(a, result_cpu, n);
    printf("result cpu: %f, time cpu: %f\n", result_cpu[0], time_cpu);

    check(result_cpu, result_gpu, 0.000001);

    return 0;

}

result gpu: 1072621.317247, time gpu: 0.000099
result cpu: 1072621.317247, time cpu: 0.003187



n=1000000

time cpu: 0.002952


**С дебагом**

time gpu: 0.003275


**Без дебага**

time gpu: 0.000638


**Без перевыделения памяти**

time gpu: 0.000152


**Развертка первого цикла и микрооптимизаации**

time gpu: 0.000099


